In [2]:
from transformers import AutoTokenizer
from huggingface_hub import login
import json
from dotenv import load_dotenv
import os

/home/lisa-linux/stuff/ma_automated_documentation/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
load_dotenv(override=True)

# log to huggingface
HF_TOKEN = os.environ['HUGGINGFACE']
login(HF_TOKEN)

In [ ]:
# full_prompt = '''
#     This repository is a Python script that fetches and generates a list of most popular repositories on GitHub based on the given programming language. The script uses the GitHub API to retrieve the required information and stores the access token in a local file named \"token.json\". The script supports multiple programming languages and can fetch up to 10 pages of results per language.\n\nThe script defines a class `RepositoryInformationProvider` that initializes a `requests.Session` object with retries and rate limit handling. It also defines methods to get the next page of results for a given language and to get the last commit date for a given repository.\n\nThe `generate_readme` function generates a markdown file with a table of most popular repositories for a given language. It fetches the data using the `RepositoryInformationProvider` and formats the data into a markdown table.\n\nThe script uses several constants and variables to store the API URLs, headers, and other configuration options. It also defines some helper functions for formatting and humanizing dates.\n\nThe script uses the `argparse` module to parse command-line arguments and supports specifying multiple languages using a comma-separated list.\n\nThe main components of the script are:\n\n* `RepositoryInformationProvider` class for fetching and handling GitHub API responses\n* `generate_readme` function for generating the markdown file\n* Use of `requests` library for making HTTP requests\n* Use of `argparse` module for parsing command-line arguments\n* Use of `json`, `time`, `humanize`, `datetime`, and `argparse` modules for various utility functions\n\nThe dependencies of the script are:\n\n* `requests` library for making HTTP requests\n* `argparse` module for parsing command-line arguments\n* `json` module for parsing JSON responses\n* `time` module for handling time-related functionality\n* `humanize` module for formatting dates\n* `datetime` module for parsing and manipulating dates\n\nThe overall architecture of the script is simple and modular, with clear separation of concerns between fetching data from the API and generating the markdown file. The script is well-documented with clear variable and function names, making it easy to understand and maintain.
# '''

In [ ]:

# def split_prompt(prompt, max_tokens):
#     tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B")
#     tokenized_prompt = tokenizer.encode(prompt)
    
#     chunks = [tokenized_prompt[i:i+max_tokens] for i in range(0, len(tokenized_prompt), max_tokens)] # (generated with Microsoft Copilot)
#     return [tokenizer.decode(chunk) for chunk in chunks] # (generated with Microsoft Copilot)


# sub_prompts = split_prompt(full_prompt, 20)


In [3]:
def create_subprompts(prompt):
    '''
    Function creates multiple chunks from given prompt to prevent limits of input tokens of choosed LLM.

    Args:
        prompt: Input which should be tokenized.
    
    Return:
        decoded_chunks
    '''
    max_tokens = 100000 # max number of tokens is 128000, but Snowflake requieres tokens for processing (see estimate_tokens())
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B") # load Autotokenizer.from_pretrained() from transformers with llm specification
    tokenized_prompt = tokenizer.encode(prompt) # encode prompt into a list of tokens
    print('Step create_subprompts: Tokens are encoded.')

    # iterates over tokenized_prompts and split content into smaller chunks (each chunks contains a max. of 124000 tokens)
    # start at i = 0 and get a chunk until max_tokens (stepsize) from tokenized_prompt
    # next chunk --> previous chunk + max_tokens
    chunks = [tokenized_prompt[i:i+max_tokens] for i in range(0, len(tokenized_prompt), max_tokens)] # (generated with Microsoft Copilot)
    decoded_chunks = [tokenizer.decode(chunk) for chunk in chunks] # decode each chunk to text # (generated with Microsoft Copilot)
    print('Step create_subprompts: Tokens are decoded.')

    return decoded_chunks

In [4]:
def create_chunks(input):
    max_tokens = 120000 # 1 token ~ 4 character
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B")
    tokenized_input = tokenizer.encode(input)
    #chunks = [tokenized_prompt[i:i+min(max_tokens, len(tokenized_prompt)-i)] for i in range(0, len(tokenized_prompt), max_tokens)]
    chunks = [tokenized_input[i:i+max_tokens] for i in range(0, len(tokenized_input), max_tokens)]
    decoded_chunks = [tokenizer.decode(chunk) for chunk in chunks]

    return decoded_chunks

In [5]:
with open('../data/input_readme_data/facebookresearch_Detectron.json', 'r') as f:
    loaded_data = json.load(f)

In [6]:
len(loaded_data['source_code_cleaned_comments'])

502686

In [7]:
max_sequence_length = 131072
#max_sequence_length = 9

In [60]:
test = 'thisismytest1234testsuperdupie89testititvonlisi'

In [58]:
len(test)

20

In [8]:
data = loaded_data['source_code_cleaned_comments']
#data = test
input = [data[i:i+max_sequence_length] for i in range(0, len(data), max_sequence_length)]


In [9]:
input

['#Copyright(c)2017-present,Facebook,Inc.##LicensedundertheApacheLicense,Version2.0(the"License");#youmaynotusethisfileexceptincompliancewiththeLicense.#YoumayobtainacopyoftheLicenseat####Unlessrequiredbyapplicablelaworagreedtoinwriting,software#distributedundertheLicenseisdistributedonan"ASIS"BASIS,#WITHOUTWARRANTIESORCONDITIONSOFANYKIND,eitherexpressorimplied.#SeetheLicenseforthespecificlanguagegoverningpermissionsand#limitationsundertheLicense.##############################################################################from__future__importabsolute_importfrom__future__importdivisionfrom__future__importprint_functionfromCython.BuildimportcythonizefromsetuptoolsimportExtensionfromsetuptoolsimportsetupimportnumpyasnp_NP_INCLUDE_DIRS=np.get_include()#Extensionmodulesext_modules=[Extension(name=\'detectron.utils.cython_bbox\',sources=[\'detectron/utils/cython_bbox.pyx\'],extra_compile_args=[\'-Wno-cpp\'],include_dirs=[_NP_INCLUDE_DIRS]),Extension(name=\'detectron.utils.cython_nms\',sourc

In [10]:
chunk_list = []

for sequence in input:
    sequence_chunks = create_chunks(sequence)
    chunk_list.append(sequence_chunks)

In [11]:
chunk_list_cleaned = [''.join(i).replace('<|begin_of_text|>', '') for i in chunk_list]

In [12]:
chunk_list_cleaned

['#Copyright(c)2017-present,Facebook,Inc.##LicensedundertheApacheLicense,Version2.0(the"License");#youmaynotusethisfileexceptincompliancewiththeLicense.#YoumayobtainacopyoftheLicenseat####Unlessrequiredbyapplicablelaworagreedtoinwriting,software#distributedundertheLicenseisdistributedonan"ASIS"BASIS,#WITHOUTWARRANTIESORCONDITIONSOFANYKIND,eitherexpressorimplied.#SeetheLicenseforthespecificlanguagegoverningpermissionsand#limitationsundertheLicense.##############################################################################from__future__importabsolute_importfrom__future__importdivisionfrom__future__importprint_functionfromCython.BuildimportcythonizefromsetuptoolsimportExtensionfromsetuptoolsimportsetupimportnumpyasnp_NP_INCLUDE_DIRS=np.get_include()#Extensionmodulesext_modules=[Extension(name=\'detectron.utils.cython_bbox\',sources=[\'detectron/utils/cython_bbox.pyx\'],extra_compile_args=[\'-Wno-cpp\'],include_dirs=[_NP_INCLUDE_DIRS]),Extension(name=\'detectron.utils.cython_nms\',sourc

In [82]:
chunk_list

[['<|begin_of_text|>#Copyright(c)2017-present,Facebook,Inc.##LicensedundertheApacheLicense,Version2.0(the"License");#youmaynotusethisfileexceptincompliancewiththeLicense.#YoumayobtainacopyoftheLicenseat####Unlessrequiredbyapplicablelaworagreedtoinwriting,software#distributedundertheLicenseisdistributedonan"ASIS"BASIS,#WITHOUTWARRANTIESORCONDITIONSOFANYKIND,eitherexpressorimplied.#SeetheLicenseforthespecificlanguagegoverningpermissionsand#limitationsundertheLicense.##############################################################################from__future__importabsolute_importfrom__future__importdivisionfrom__future__importprint_functionfromCython.BuildimportcythonizefromsetuptoolsimportExtensionfromsetuptoolsimportsetupimportnumpyasnp_NP_INCLUDE_DIRS=np.get_include()#Extensionmodulesext_modules=[Extension(name=\'detectron.utils.cython_bbox\',sources=[\'detectron/utils/cython_bbox.pyx\'],extra_compile_args=[\'-Wno-cpp\'],include_dirs=[_NP_INCLUDE_DIRS]),Extension(name=\'detectron.utils.

In [64]:
input

['thisismyt', 'est1234te', 'stsuperdu', 'pie89test', 'ititvonli', 'si']

In [52]:
input

['thisi', 'smyte', 'st123', '4text', '0']

In [55]:
input

['thisi', 'smyte', 'st123', '4te']

In [36]:
list_with_chunks = create_subprompts(input)

In [37]:
len(list_with_chunks)

1